Дана статистика пользователей adult.csv.

Получите значения AUC для различных моделей и их параметров.

Решением будет ссылка на гитхаб с ноутбуком.

In [61]:
import pandas as pd
import numpy as np

In [62]:
data = pd.read_csv('adult.csv')
data.head()



,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [63]:

data.info()
# Ok

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [65]:
# Run preprocessing
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Create some toy data in a Pandas dataframe
# fruit_data = pd.DataFrame({
#     'fruit':  ['apple','orange','pear','orange'],
#     'color':  ['red','orange','green','green'],
#     'weight': [5,6,3,4]
# })

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [70]:
# Feature ingineering
# data.drop(data[data['workclass'] == '?'].index, inplace = True)
# lebal_encoder
columns_for_le = ('workclass', 'education', 'marital-status', 'occupation', 'relationship', 
                  'race', 'gender', 'native-country', 'income')

# run le
data_le = MultiColumnLabelEncoder(columns = columns_for_le).fit_transform(data)
data_le.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,4,226802,1,7,4,7,3,2,1,0,0,40,39,0
1,38,4,89814,11,9,2,5,0,4,1,0,0,50,39,0
2,28,2,336951,7,12,2,11,0,4,1,0,0,40,39,1
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39,1
4,18,0,103497,15,10,4,0,3,4,0,0,0,30,39,0


In [78]:
# Choose algoritms which I'll use
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

algorithms = [
    LogisticRegression(),
    SVC(),
    SGDClassifier(),
    RandomForestClassifier()
]

In [71]:
# split data
import numpy as np
from sklearn.model_selection import train_test_split

X = data_le.loc[:, data.columns !='income']
y = data_le[['income']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 42 - The Ultimate Question of Life, the Universe, and Everything
X_train.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
42392,25,4,188767,11,9,4,3,3,4,1,0,0,40,39
14623,64,4,286732,11,9,6,12,1,4,0,0,0,17,39
27411,29,4,253801,11,9,2,3,0,4,1,0,0,40,7
1288,28,4,334032,8,11,2,3,0,4,1,0,0,50,39
7078,22,4,173004,11,9,4,7,2,2,1,0,0,1,39


In [80]:
# for algoritm in algorithms:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'{algoritm} - accuracy: {accuracy}')

rf_auc = roc_auc_score(y_test, rfc.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print("AUC для случайного леса: {:.3f}".format(rf_auc))
# print("AUC для SVC: {:.3f}".format(svc_auc))



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) - accuracy: 0.8615300621033235
AUC для случайного леса: 0.911


In [84]:
# run all algoritms
for algoritm in algorithms:
    model = algoritm.fit(X_train, y_train)
    model_name = type(model).__name__
    y_pred = algoritm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print(f'{model_name} -> accuracy: {accuracy}, algoritm_s AUC: {AUC}')

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), f

LogisticRegression -> accuracy: 0.7939671057121409, algoritm_s AUC: 0.6250274228554858
SVC -> accuracy: 0.8041356718760664, algoritm_s AUC: 0.5827481605732483


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier -> accuracy: 0.7974476216474442, algoritm_s AUC: 0.6211962924611747


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier -> accuracy: 0.8648058418071385, algoritm_s AUC: 0.7866454598319592


In [120]:
algorithms1['SVC']

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [122]:
# GridSearchCV - find the best hiperparameters for every algoritms
from sklearn.model_selection import GridSearchCV

# var1
# param_grid = [
#  {'classifier':['LogisticRegression()'], 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100], 
#   'classifier__penalty': ['l1','l2', 'elasticnet']},   
#  {'classifier': ['SVC()'], 'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
#  'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
#  {'classifier':['SGDClassifier()'], 'classifier__max_iter': [5, 10, 20], 
#   'classifier__penalty': ['l1','l2', 'elasticnet']},  
#  {'classifier': ['RandomForestClassifier()'],'classifier__max_features': [1, 2, 3],
#  'classifier__n_estimators': [5, 50, 100]}
# ]


# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


# var2
# param_grid = {'LogisticRegression__C': [0.001, 0.01, 0.1, 1, 10, 100], 
#   'LogisticRegression__penalty': ['l1','l2', 'elasticnet'], 
#   'SVC__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
#   'SVC__C': [0.001, 0.01, 0.1, 1, 10, 100],
#   'SGDClassifier__max_iter': [5, 10, 20], 
#   'SGDClassifier__penalty': ['l1','l2', 'elasticnet'],  
#   'RandomForestClassifier__max_features': [1, 2, 3],
#   'RandomForestClassifier__n_estimators': [5, 50, 100]
#  }


# var 3
algorithms1 = {
    'LogisticRegression': LogisticRegression(),
    'SVC':SVC(),
    'SGDClassifier':SGDClassifier(),
    'RandomForestClassifier':RandomForestClassifier()
}

param_grid = [
 {'classifier':['LogisticRegression'], 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100], 
  'classifier__penalty': ['l1','l2', 'elasticnet']},   
 {'classifier': ['SVC'], 'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
 {'classifier':['SGDClassifier'], 'classifier__max_iter': [5, 10, 20], 
  'classifier__penalty': ['l1','l2', 'elasticnet']},  
 {'classifier': ['RandomForestClassifier'],'classifier__max_features': [1, 2, 3],
 'classifier__n_estimators': [5, 50, 100]}
]

# parameters for SVC
# param_grid = {'C': [  0.1, 1, 10, ],
#  'gamma': [ 0.1, 1, 10]}


# param_grid

In [124]:
# check on one model before expanding to other models
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)


ValueError: Invalid parameter classifier for estimator SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [116]:


for algoritm in algorithms1:
    print(algoritm)

LogisticRegression
SVC
SGDClassifier
RandomForestClassifier
